# Data voorbewerking

De data voor dit project is afkomstig van het back office systeem van Financial Markets, deels aangevuld met extra data vanuit Bloomberg.
De data bestaat uit de volgende elementen:

- Bond data
- Bondprijzen
- Government Yield curves
- Inflation data

Alle data is extracted en opgeslagen in csv files. In dit workbook lopen we door de data voorbereiding heen. Alle hier genoemde stappen kunnen ook geautomatiseerd worden uitgevoerd door het shell command 'Make Data'.

In [1]:
%load_ext autoreload
%autoreload 2

import sys
import pandas as pd

sys.path.insert(0, "..") 
from src.data import make_dataset
from src.features import build_features

# Bond data

In [2]:

# Get bond data, drop unneeded columns, convert formats and strip training blanks
df_bonds = make_dataset.get_bond_data()

2022-01-03 10:33:49.417 | INFO     | src.data.make_dataset:get_bond_data:35 - Load bond data
2022-01-03 10:33:49.418 | INFO     | src.data.make_dataset:read_csv:20 - Loading data from ..\data\raw\bonds.csv


In [3]:
df_bonds.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 225 entries, 0 to 224
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   ccy                225 non-null    string        
 1   country            225 non-null    string        
 2   issue_dt           225 non-null    datetime64[ns]
 3   first_coupon_date  202 non-null    datetime64[ns]
 4   mature_dt          225 non-null    datetime64[ns]
 5   isin               225 non-null    string        
 6   issuer_name        225 non-null    string        
 7   coupon_frq         225 non-null    string        
 8   coupon             225 non-null    float64       
 9   tot_issue          225 non-null    float64       
 10  cfi_code           218 non-null    string        
 11  issue_rating       220 non-null    string        
dtypes: datetime64[ns](3), float64(2), string(7)
memory usage: 21.2 KB


In [4]:
df_bonds.head()

,ccy,country,issue_dt,first_coupon_date,mature_dt,isin,issuer_name,coupon_frq,coupon,tot_issue,cfi_code,issue_rating
0,EUR,Netherlands,2009-02-13,2009-07-15,2019-07-15,NL0009086115,STAAT DER NEDERLANDEN,ANNUAL,4.000,5.000000e+09,DBFTFN,AAA
1,NLG,Austria,1994-02-28,1995-02-28,2024-02-28,NL0000133924,AUSTRIA,ANNUAL,6.250,1.000000e+09,DBFTXB,AA+
2,EUR,Netherlands,2012-03-09,2013-01-15,2033-01-15,NL0010071189,STAAT DER NEDERLANDEN,ANNUAL,2.500,4.160000e+09,DBFXXN,AAA
3,USD,United States,2009-05-15,2009-11-15,2019-05-15,US912828KQ20,UNITED STATES TREASURY,SEMI ANNUAL,3.125,6.441100e+10,<NA>,<NA>
4,USD,United States,2010-02-15,2010-08-15,2020-02-15,US912828MP29,UNITED STATES TREASURY,SEMI ANNUAL,3.625,0.000000e+00,<NA>,<NA>


Imputeren ontbrekende waarden voor issue rating. 
Issue rating wordt waar deze ontbreekt ingevuld met de meest voorkomende issue rating voor de issuer.

CFI code wordt waar deze ontbreekt ingevuld met code 'onbekend' = DXXXXX.

Datums met de waarde 1899-12-30 zijn default waardes van het bronsysteem. Deze worden verwijderd.
Daar waar de eerste coupon datum ontbreekt (o.a. zero coupon bonds) - wordt deze aangevuld met de issue datum.
De reden hiervoor is dat we zo zonder veel moeite de looptijd van de bond kunnen berekenen.

In [5]:
df_bonds = make_dataset.impute_bonds(df_bonds)

2022-01-03 10:33:49.868 | INFO     | src.data.make_dataset:impute_bonds:68 - Impute bond data


In [6]:
make_dataset.save_pkl('bonds', df_bonds)

2022-01-03 10:33:49.990 | INFO     | src.data.make_dataset:save_pkl:339 - Save preprocessed bonds data


In [7]:
df_bonds.count()

ccy                  225
country              225
issue_dt             225
first_coupon_date    225
mature_dt            225
isin                 225
issuer_name          225
coupon_frq           225
coupon               225
tot_issue            225
cfi_code             225
issue_rating         225
bond_duration        225
dtype: int64

# Bondprijzen

In [8]:
df_price = make_dataset.get_price()

2022-01-03 10:33:50.216 | INFO     | src.data.make_dataset:get_price:105 - Load bond price data
2022-01-03 10:33:50.217 | INFO     | src.data.make_dataset:read_csv:20 - Loading data from ..\data\raw\price.csv


In [9]:
df_price = make_dataset.impute_price(df_price)

2022-01-03 10:33:50.843 | INFO     | src.data.make_dataset:impute_price:130 - Impute bond price


In [10]:
df_price.head()

,reference_identifier,ccy,rate_dt,mid
0,BE0000332412,EUR,2014-01-20,100.719
1,BE0000332412,EUR,2014-01-24,101.005
2,BE0000332412,EUR,2014-01-28,100.953
3,BE0000332412,EUR,2014-01-22,100.359
4,BE0000332412,EUR,2014-01-21,100.601


In [11]:
df_price.describe()

,mid
count,209276.000000
mean,111.289025
std,15.522802
min,84.429000
25%,102.285000
50%,106.729000
75%,112.926000
max,195.749000


In [12]:
df_price.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 209276 entries, 0 to 220236
Data columns (total 4 columns):
 #   Column                Non-Null Count   Dtype         
---  ------                --------------   -----         
 0   reference_identifier  209276 non-null  string        
 1   ccy                   209276 non-null  string        
 2   rate_dt               209276 non-null  datetime64[ns]
 3   mid                   209276 non-null  float64       
dtypes: datetime64[ns](1), float64(1), string(2)
memory usage: 8.0 MB


In [13]:
make_dataset.save_pkl('price', df_price)

2022-01-03 10:33:51.406 | INFO     | src.data.make_dataset:save_pkl:339 - Save preprocessed price data


# Government Yield curves

In [14]:
df_yield = make_dataset.get_yield()

2022-01-03 10:33:54.521 | INFO     | src.data.make_dataset:get_yield:138 - Load goverment yield curve data
2022-01-03 10:33:54.522 | INFO     | src.data.make_dataset:read_csv:20 - Loading data from ..\data\raw\yield.csv


In [15]:
df_yield = make_dataset.impute_yield(df_yield)

2022-01-03 10:33:55.857 | INFO     | src.data.make_dataset:impute_yield:167 - Impute yield curve


In [16]:
df_yield.tail()

,country,rate_dt,timeband,ratename,ccy,actual_dt,datedays,bid,offer,int_basis,time
116809,Spain,2021-12-23,5 YEARS,GOV Yield Curve ES BB,EUR,2026-12-23,1831,-0.062,-0.081,ANNUAL,1826 days
116811,Spain,2021-12-23,6 YEARS,GOV Yield Curve ES BB,EUR,2027-12-23,2195,-0.015,-0.030,ANNUAL,2191 days
116812,Spain,2021-12-23,7 YEARS,GOV Yield Curve ES BB,EUR,2028-12-23,2561,0.071,0.057,ANNUAL,2557 days
116813,Spain,2021-12-23,8 YEARS,GOV Yield Curve ES BB,EUR,2029-12-23,2926,0.240,0.229,ANNUAL,2922 days
116815,Spain,2021-12-23,9 YEARS,GOV Yield Curve ES BB,EUR,2030-12-23,3291,0.367,0.357,ANNUAL,3287 days


In [17]:
df_yield.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 89750 entries, 1 to 116815
Data columns (total 11 columns):
 #   Column     Non-Null Count  Dtype          
---  ------     --------------  -----          
 0   country    89750 non-null  object         
 1   rate_dt    89750 non-null  datetime64[ns] 
 2   timeband   89750 non-null  object         
 3   ratename   89750 non-null  string         
 4   ccy        89750 non-null  string         
 5   actual_dt  89750 non-null  datetime64[ns] 
 6   datedays   89750 non-null  int64          
 7   bid        89750 non-null  float64        
 8   offer      89750 non-null  float64        
 9   int_basis  89750 non-null  string         
 10  time       89750 non-null  timedelta64[ns]
dtypes: datetime64[ns](2), float64(2), int64(1), object(2), string(3), timedelta64[ns](1)
memory usage: 8.2+ MB


In [18]:
make_dataset.save_pkl('yield', df_yield)

2022-01-03 10:33:56.460 | INFO     | src.data.make_dataset:save_pkl:339 - Save preprocessed yield data


# Inflation data


In [19]:
df_inflation = make_dataset.get_inflation()  

2022-01-03 10:34:00.260 | INFO     | src.data.make_dataset:get_inflation:187 - Load goverment yield curve data
2022-01-03 10:34:00.261 | INFO     | src.data.make_dataset:read_csv:20 - Loading data from ..\data\raw\DE Inflation.csv
2022-01-03 10:34:00.282 | INFO     | src.data.make_dataset:read_csv:20 - Loading data from ..\data\raw\FR Inflation.csv
2022-01-03 10:34:00.300 | INFO     | src.data.make_dataset:read_csv:20 - Loading data from ..\data\raw\ES Inflation.csv
2022-01-03 10:34:00.317 | INFO     | src.data.make_dataset:read_csv:20 - Loading data from ..\data\raw\IT Inflation.csv
2022-01-03 10:34:00.333 | INFO     | src.data.make_dataset:read_csv:20 - Loading data from ..\data\raw\US Inflation.csv


In [20]:
df_inflation = make_dataset.impute_inflation(df_inflation)


df_inflation.info()

2022-01-03 10:34:15.094 | INFO     | src.data.make_dataset:impute_inflation:215 - Impute inflation curve


<class 'pandas.core.frame.DataFrame'>
Int64Index: 215007 entries, 0 to 216873
Data columns (total 7 columns):
 #   Column     Non-Null Count   Dtype          
---  ------     --------------   -----          
 0   country    215007 non-null  string         
 1   rate_dt    215007 non-null  datetime64[ns] 
 2   timeband   215007 non-null  string         
 3   inflation  215007 non-null  float64        
 4   ratename   215007 non-null  string         
 5   actual_dt  215007 non-null  datetime64[ns] 
 6   time       215007 non-null  timedelta64[ns]
dtypes: datetime64[ns](2), float64(1), string(3), timedelta64[ns](1)
memory usage: 13.1 MB


In [21]:
df_inflation.head()

,country,rate_dt,timeband,inflation,ratename,actual_dt,time
0,Germany,2021-12-23,1 YEAR,3.28625,Inflation,2022-12-23,365 days
1,Germany,2021-12-22,1 YEAR,3.33875,Inflation,2022-12-22,365 days
2,Germany,2021-12-21,1 YEAR,3.15625,Inflation,2022-12-21,365 days
3,Germany,2021-12-20,1 YEAR,3.01375,Inflation,2022-12-20,365 days
4,Germany,2021-12-17,1 YEAR,2.89875,Inflation,2022-12-17,365 days


In [22]:
make_dataset.save_pkl('inflation', df_inflation)

2022-01-03 10:34:15.736 | INFO     | src.data.make_dataset:save_pkl:339 - Save preprocessed inflation data


In [23]:
#df_bpi = make_dataset.join_inflation(df_bonds, df_price, df_inflation)
#df_bpy = make_dataset.join_yield(df_bonds, df_price, df_yield)

# Country Spread

In [24]:
df_yield

,country,rate_dt,timeband,ratename,ccy,actual_dt,datedays,bid,offer,int_basis,time
1,France,2016-06-01,1 YEAR,GOV Yield Curve FR BB,EUR,2017-06-01,369,-0.5200,-0.498,ANNUAL,365 days
2,France,2016-06-01,10 YEARS,GOV Yield Curve FR BB,EUR,2026-06-01,3654,0.4755,0.475,ANNUAL,3652 days
3,France,2016-06-01,15 YEARS,GOV Yield Curve FR BB,EUR,2031-06-02,5480,0.4755,0.475,ANNUAL,5479 days
4,France,2016-06-01,2 YEARS,GOV Yield Curve FR BB,EUR,2018-06-01,733,-0.4597,-0.442,ANNUAL,730 days
5,France,2016-06-01,20 YEARS,GOV Yield Curve FR BB,EUR,2036-06-01,7307,1.2151,1.220,ANNUAL,7305 days
...,...,...,...,...,...,...,...,...,...,...,...
116809,Spain,2021-12-23,5 YEARS,GOV Yield Curve ES BB,EUR,2026-12-23,1831,-0.0620,-0.081,ANNUAL,1826 days
116811,Spain,2021-12-23,6 YEARS,GOV Yield Curve ES BB,EUR,2027-12-23,2195,-0.0150,-0.030,ANNUAL,2191 days
116812,Spain,2021-12-23,7 YEARS,GOV Yield Curve ES BB,EUR,2028-12-23,2561,0.0710,0.057,ANNUAL,2557 days
116813,Spain,2021-12-23,8 YEARS,GOV Yield Curve ES BB,EUR,2029-12-23,2926,0.2400,0.229,ANNUAL,2922 days


In [30]:
df_yield[df_yield['country']=='Netherlands']

,country,rate_dt,timeband,ratename,ccy,actual_dt,datedays,bid,offer,int_basis,time
72670,Netherlands,2016-06-01,10 YEARS,GOV Yield Curve NL BB,EUR,2026-06-01,3654,0.3440,0.3470,ANNUAL,3652 days
72671,Netherlands,2016-06-01,2 YEARS,GOV Yield Curve NL BB,EUR,2018-06-01,733,-0.4822,-0.4505,ANNUAL,730 days
72672,Netherlands,2016-06-01,20 YEARS,GOV Yield Curve NL BB,EUR,2036-06-01,7307,0.7436,0.7549,ANNUAL,7305 days
72674,Netherlands,2016-06-01,3 YEARS,GOV Yield Curve NL BB,EUR,2019-06-02,1097,-0.4651,-0.4357,ANNUAL,1096 days
72675,Netherlands,2016-06-01,30 YEARS,GOV Yield Curve NL BB,EUR,2046-06-01,10960,0.9598,0.9701,ANNUAL,10957 days
...,...,...,...,...,...,...,...,...,...,...,...
92605,Netherlands,2021-12-23,5 YEARS,GOV Yield Curve NL BB,EUR,2026-12-23,1831,-0.5040,-0.5100,ANNUAL,1826 days
92607,Netherlands,2021-12-23,6 YEARS,GOV Yield Curve NL BB,EUR,2027-12-23,2195,-0.4280,-0.4350,ANNUAL,2191 days
92608,Netherlands,2021-12-23,7 YEARS,GOV Yield Curve NL BB,EUR,2028-12-23,2561,-0.3620,-0.3660,ANNUAL,2557 days
92609,Netherlands,2021-12-23,8 YEARS,GOV Yield Curve NL BB,EUR,2029-12-23,2926,-0.2640,-0.2670,ANNUAL,2922 days


In [26]:
# df_countryspread = build_features.country_spread(df_yield)
#df_countryspread['timeband'] = df_countryspread['timeband'].str.replace('YEARS','').str.replace('YEAR','')

#df_countryspread_pivot = pd.pivot(df_countryspread, index = ['rate_dt'], columns = ['timeband'])
#df_countryspread_pivot

#$= [''.join(('yield_',''.join(tup))).replace('YEARS','').replace('YEAR','').strip() for tup in df_yield_pivot.columns]
#$    columns = df_yield_pivot.columns.to_list()
#$    columns.sort(key=lambda x: int(x[6:].replace('bid','').replace('offer','')))
#$    df_yield_pivot = df_yield_pivot[columns]
#df_countryspread_pivot.columns = [' '.join(col).replace('YEARS','').replace('YEAR','').strip() for col in df_countryspread_pivot.columns.values]
#columns = df_countryspread_pivot.columns.to_list()
#columns.sort(key=lambda x: int(x[6:].replace('cs_','')))
#df_countryspread_pivot = df_countryspread_pivot[columns]

2022-01-03 10:34:30.023 | INFO     | src.features.build_features:country_spread:49 - Add Country Spread


In [27]:
df_countryspread.head()

,rate_dt,timeband,actual_dt,time,cs_FRDE,cs_FRIT,cs_FRNL,cs_FRES,cs_DEIT,cs_DENL,cs_DEES,cs_ITNL,cs_ITES,cs_NLES
0,2016-06-01,1 YEAR,2017-06-01,365 days,0.021,-0.325,NaN,-0.285,-0.346,NaN,-0.306,NaN,0.040,NaN
1,2016-06-01,10 YEARS,2026-06-01,3652 days,0.341,-0.916,0.1280,-1.023,-1.257,-0.2130,-1.364,1.0440,-0.107,-1.1510
2,2016-06-01,15 YEARS,2031-06-02,5479 days,0.341,-0.916,NaN,-1.023,-1.257,NaN,-1.364,NaN,-0.107,NaN
3,2016-06-01,2 YEARS,2018-06-01,730 days,0.073,-0.382,0.0085,-0.351,-0.455,-0.0645,-0.424,0.3905,0.031,-0.3595
4,2016-06-01,20 YEARS,2036-06-01,7305 days,0.564,-0.933,0.4651,-1.158,-1.497,-0.0989,-1.722,1.3981,-0.225,-1.6231


In [ ]:
df_bpiy = make_dataset.fulljoin(df_bonds, df_price, df_inflation, df_yield, df_countryspread)
df_bpiy = build_features.add_duration(df_bpiy)
df_bpiy = build_features.add_bid_offer_spread(df_bpiy)

df_bpy.to_csv(f'../data/processed/bpy.csv')
df_bpi.to_csv(f'../data/processed/bpi.csv')
df_bpiy.to_csv(f'../data/processed/bpiy.csv')

In [ ]:
df_pivot1 = df_pivot.drop(['bid','offer'], axis=1, level=0)
df_pivot1.columns = df_pivot1.columns.droplevel(1)
df_pivot1.reset_index()

In [ ]:
df_bpiy

All in one make statement...

In [ ]:
make_dataset.make_data()